In [1]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, T5Tokenizer, T5ForConditionalGeneration
import torch
import pandas as pd

2025-08-31 17:58:35.329431: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
def make_predictions(model, tokenizer, input_text, prompt, device, encoding="utf-8"):
    predictions = []
    for line in input_text:
        inputs = tokenizer.encode(prompt + line, return_tensors="pt").to(device)
        outputs = model.generate(inputs)
        prediction = tokenizer.decode(outputs[0])
        predictions.append(prediction)

    return predictions

In [3]:
def format_predictions(predictions_text):

    extracted_lines = [line[line.find('<pad>') + 5 : line.find('</s>')].strip() for line in predictions_text]

    return extracted_lines


In [4]:
from sklearn.metrics import classification_report, f1_score, accuracy_score
def calculate_accuracy(predictions, target):

    labels = [line.strip() for line in target]

    predictions = [line.strip() for line in predictions]

    # Compare the predictions with the actual labels and calculate the accuracy
    print(classification_report(labels, predictions))
    accuracy = accuracy_score(labels, predictions)
    print(f"Accuracy: {accuracy * 100}%")
    print(f"Macro F1 score: {f1_score(labels, predictions)}")

In [5]:
df_test = pd.read_csv("NLP_ass_test.tsv", sep="\t", names=["text", "label"])

test_text = df_test["text"].values.tolist()
test_labels = df_test["label"].values.tolist()

# Zero shot prompting

In [6]:
# Load the FLAN T5 model and tokenizer
tokenizer_base = T5Tokenizer.from_pretrained("google/flan-t5-base")
model_base = T5ForConditionalGeneration.from_pretrained("google/flan-t5-base")

tokenizer_small = T5Tokenizer.from_pretrained("google/flan-t5-small")
model_small = T5ForConditionalGeneration.from_pretrained("google/flan-t5-small")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
model_base.to(device)
model_small.to(device)

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


...

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

cpu


T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=384, bias=False)
              (k): Linear(in_features=512, out_features=384, bias=False)
              (v): Linear(in_features=512, out_features=384, bias=False)
              (o): Linear(in_features=384, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 6)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=512, out_features=1024, bias=False)
              (wi_1): Linear(in_features=512, out_features=1024, bias=False)
              (wo): 

## FLAN T5 Base model

In [7]:
prompt = """classify the following comment in to either "hatespeech", or "offensive" or "normal" ONLY : """

predictions_zero_base = make_predictions(model_base, tokenizer_base, test_text, prompt, device)

predictions_zero_base = format_predictions(predictions_zero_base)

In [9]:
from sklearn.metrics import classification_report, f1_score, accuracy_score
def calculate_accuracy(predictions, target):

    labels = [line.strip() for line in target]

    predictions = [line.strip() for line in predictions]

    # Compare the predictions with the actual labels and calculate the accuracy
    print(classification_report(labels, predictions))
    accuracy = accuracy_score(labels, predictions)
    print(f"Accuracy: {accuracy * 100}%")
    print(f"Macro F1 score: {f1_score(labels, predictions, average='macro')}")
calculate_accuracy(predictions_zero_base, test_labels)

              precision    recall  f1-score   support

  hatespeech       0.51      0.64      0.56       594
      normal       0.58      0.64      0.61       782
   offensive       0.33      0.19      0.24       548

    accuracy                           0.51      1924
   macro avg       0.47      0.49      0.47      1924
weighted avg       0.49      0.51      0.49      1924

Accuracy: 51.03950103950103%
Macro F1 score: 0.4700227612620078


## FLAN T5 small model

In [10]:
prompt = """classify the following comment in to either "hatespeech", or "offensive" or "normal" : """

predictions_zero_small = make_predictions(model_small, tokenizer_small, test_text, prompt, device)

predictions_zero_small = format_predictions(predictions_zero_small)

In [11]:
calculate_accuracy(predictions_zero_small, test_labels)


                  precision    recall  f1-score   support

      acceptable       0.00      0.00      0.00         0
    anti semitic       0.00      0.00      0.00         0
      hatespeech       0.25      0.01      0.01       594
      hypocrites       0.00      0.00      0.00         0
        nonsense       0.00      0.00      0.00         0
          normal       0.50      0.01      0.01       782
       offensive       0.29      0.99      0.45       548
      stereotype       0.00      0.00      0.00         0
stereotypespeech       0.00      0.00      0.00         0
unpopularopinion       0.00      0.00      0.00         0
     unrepentant       0.00      0.00      0.00         0

        accuracy                           0.29      1924
       macro avg       0.09      0.09      0.04      1924
    weighted avg       0.36      0.29      0.14      1924

Accuracy: 28.586278586278585%
Macro F1 score: 0.042534971731280846


/home/tony/miniconda3/envs/nlp-a2/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/tony/miniconda3/envs/nlp-a2/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/tony/miniconda3/envs/nlp-a2/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


# Few shot prompting

## FLAN T5 base model

In [12]:
prompt = """Hatespeech is defined as sentences that attack or degrade a person or group on the basis of attributes such as race, religion, ethnic origin, national origin, sex, disability, sexual orientation, or gender identity. Normal sentences describe everyday occurrences, opinions, beliefs, or feelings without attacking or degrading others. Offensive sentences contain profanity, insults, or negative statements intended to offend, but not specifically targeting protected groups or containing threats.

Classify the following sentence into either 'hatespeech' or 'offensive' or 'normal':

'Today is a nice day': 'normal'
'You are stupid': 'offensive'
'I don't like green cars': 'normal'
'Fuck you, idiot': 'offensive'
'a negro moslem doctor got struck off for negligence when a white <number> year old kid with down syndrome died the guardians response is to help promote a crowd funding campaign for her': 'hatespeech'
'anyway princess sapphire of silverland is a dyke and threw the first brick at stonewall': 'offensive'
'if we could teach a giraffe to paint that would be one handy sumbitch': 'normal'
'she wanna meet carti that bitch is a barbie 🦋': 'normal'
'smiling cuz i am young black rich and i am handsome': 'normal'
'bitches be so fine and got the nerve to like niggas 😒': 'offensive'

Sentence: """

predictions_few_base = make_predictions(model_base, tokenizer_base, test_text, prompt, device)

predictions_few_base = format_predictions(predictions_few_base)

Token indices sequence length is longer than the specified maximum sequence length for this model (534 > 512). Running this sequence through the model will result in indexing errors


In [13]:
calculate_accuracy(predictions_few_base, test_labels)

              precision    recall  f1-score   support

  hatespeech       0.67      0.35      0.46       594
      normal       0.50      0.83      0.62       782
   offensive       0.39      0.21      0.27       548

    accuracy                           0.51      1924
   macro avg       0.52      0.46      0.45      1924
weighted avg       0.52      0.51      0.47      1924

Accuracy: 50.67567567567568%
Macro F1 score: 0.45122897067014717


## FLAN T5 small model

In [14]:
prompt = """Hatespeech is defined as sentences that attack or degrade a person or group on the basis of attributes such as race, religion, ethnic origin, national origin, sex, disability, sexual orientation, or gender identity. Normal sentences describe everyday occurrences, opinions, beliefs, or feelings without attacking or degrading others. Offensive sentences contain profanity, insults, or negative statements intended to offend, but not specifically targeting protected groups or containing threats.

Classify the following sentence into either 'hatespeech' or 'offensive' or 'normal':

'Today is a nice day': 'normal'
'You are stupid': 'offensive'
'I don't like green cars': 'normal'
'Fuck you, idiot': 'offensive'
'a negro moslem doctor got struck off for negligence when a white <number> year old kid with down syndrome died the guardians response is to help promote a crowd funding campaign for her': 'hatespeech'
'anyway princess sapphire of silverland is a dyke and threw the first brick at stonewall': 'offensive'
'if we could teach a giraffe to paint that would be one handy sumbitch': 'normal'
'she wanna meet carti that bitch is a barbie 🦋': 'normal'
'smiling cuz i am young black rich and i am handsome': 'normal'
'bitches be so fine and got the nerve to like niggas 😒': 'offensive'

Sentence: """

predictions_few_small = make_predictions(model_small, tokenizer_small, test_text, prompt, device)

predictions_few_small = format_predictions(predictions_few_small)

Token indices sequence length is longer than the specified maximum sequence length for this model (534 > 512). Running this sequence through the model will result in indexing errors


In [16]:
predictions_few_small = make_predictions(model_small, tokenizer_small, test_text, prompt, device)

predictions_few_small = format_predictions(predictions_few_small)

calculate_accuracy(predictions_few_small, test_labels)

              precision    recall  f1-score   support

  acceptable       0.00      0.00      0.00         0
  hatespeech       0.00      0.00      0.00       594
      normal       0.00      0.00      0.00       782
   offensive       0.29      1.00      0.44       548
   unnatural       0.00      0.00      0.00         0

    accuracy                           0.28      1924
   macro avg       0.06      0.20      0.09      1924
weighted avg       0.08      0.28      0.13      1924

Accuracy: 28.482328482328484%
Macro F1 score: 0.08874493927125507


/home/tony/miniconda3/envs/nlp-a2/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/tony/miniconda3/envs/nlp-a2/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/tony/miniconda3/envs/nlp-a2/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(